#  Create a Lex Bot V2 using AWS SDK Python

Using AWS Lex V2 APIs, we can create a Lex bot from a list of utterances. We can generate this list of utterances 
from a file as well.

Author: msb.nisha@gmail.com

In [10]:
import boto3
from botocore.config import Config

In [59]:
my_config = Config(
    region_name = 'us-east-1',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [60]:
#Create Lex Bot
client = boto3.client('lexv2-models', config=my_config)
response = client.create_bot(
    botName='NI_TestOne',
    description='Test bot for development',
    roleArn= 'arn:aws:iam::604907239905:role/aws-service-role/lexv2.amazonaws.com/AWSServiceRoleForLexV2Bots', # Role name: 'AWSServiceRoleForLexV2Bots',
    dataPrivacy={
        'childDirected': False
    },
    idleSessionTTLInSeconds=123,
    botTags={
        'segment': 'analytics'
    },
   testBotAliasTags={
       'releasetype': 'nonprod'
 }
)

In [61]:
print("Request status :", response['ResponseMetadata']['HTTPStatusCode'])
print("Bot Id is      :", response['botId'])
bot_id = response['botId']

Request status : 202
Bot Id is      : HC7JOEGNKS


In [62]:
#Viewing Bot properties using bot_id
client.describe_bot(
    botId=bot_id
)


{'ResponseMetadata': {'RequestId': 'ec644a02-0c09-4043-929e-725da6708447',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 04 Sep 2021 12:07:34 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '360',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ec644a02-0c09-4043-929e-725da6708447',
   'x-amz-apigw-id': 'FIwtBGDCoAMFQ6A=',
   'x-amzn-trace-id': 'Root=1-61336186-54dbbc23514624576999d727'},
  'RetryAttempts': 0},
 'botId': 'HC7JOEGNKS',
 'botName': 'NI_TestOne',
 'description': 'Test bot for development',
 'roleArn': 'arn:aws:iam::604907239905:role/aws-service-role/lexv2.amazonaws.com/AWSServiceRoleForLexV2Bots',
 'dataPrivacy': {'childDirected': False},
 'idleSessionTTLInSeconds': 123,
 'botStatus': 'Available',
 'creationDateTime': datetime.datetime(2021, 9, 4, 17, 37, 12, 307000, tzinfo=tzlocal()),
 'lastUpdatedDateTime': datetime.datetime(2021, 9, 4, 17, 37, 12, 307000, tzinfo=tzlocal())}

In [63]:
response = client.list_bot_versions(
    botId=bot_id,
    sortBy={
        'attribute': 'BotVersion',
        'order': 'Ascending'
    },
    maxResults=123
    #nextToken='string'
)
version_name = response['botVersionSummaries'][0]['botVersion']
print("version name :", version_name)

version name : DRAFT


You must add a locale to a bot before you can add intents and slot types to the bot.

In [65]:
response = client.create_bot_locale(
    botId=bot_id,
    botVersion=version_name, #The version of the bot to create the locale for. This can only be the draft version of the bot.
    localeId='en_US',
    description='Bot locale',
    nluIntentConfidenceThreshold=0.5,
    #voiceSettings={
    #    'voiceId': 'string'
    #}
)
response

{'ResponseMetadata': {'RequestId': '603e9efd-cc11-4ed4-93b5-badcd7dcb7d3',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 04 Sep 2021 12:08:49 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '238',
   'connection': 'keep-alive',
   'x-amzn-requestid': '603e9efd-cc11-4ed4-93b5-badcd7dcb7d3',
   'x-amz-apigw-id': 'FIw4vFv1IAMFffA=',
   'x-amzn-trace-id': 'Root=1-613361d1-7784cf264e7ab7bf644ba00b'},
  'RetryAttempts': 0},
 'botId': 'HC7JOEGNKS',
 'botVersion': 'DRAFT',
 'localeName': 'English (US)',
 'localeId': 'en_US',
 'description': 'Bot locale',
 'nluIntentConfidenceThreshold': 0.5,
 'botLocaleStatus': 'Creating',
 'creationDateTime': datetime.datetime(2021, 9, 4, 17, 38, 49, 446000, tzinfo=tzlocal())}

In [66]:
#Create Intent
def create_intent(client, bot_id, version, intent_name, utterances):
    
    response = client.create_intent(
    intentName=intent_name,
    description= intent_name,
    sampleUtterances= utterances,

    dialogCodeHook={
        'enabled': False
    },
    fulfillmentCodeHook={
        'enabled': False
    },
         
    botId=bot_id,
    botVersion=version,
    localeId='en_US'
    
    )
    return (response)

Adding Intents to the bot, by calling create_intent function

In [67]:

intents_org = ['myintentone', 'myintenttwo', 'myintentthree']

for intentname in intents_org:
    sampleUtterances = [ {
        "utterance":'First Utterance for ' + intentname
        }
        
       ]
    resp = create_intent(client, bot_id, version_name, intentname, sampleUtterances )
    if resp ['ResponseMetadata']['HTTPStatusCode'] != 200:
        print("Intent Creation failed!!!\n", resp)
        break

In [70]:
#List Intents of a Bot
response = client.list_intents(
    botId=bot_id,
    botVersion=version_name,
    localeId='en_US',
    
    maxResults=123
)
response['intentSummaries']

[{'intentId': 'FALLBCKINT',
  'intentName': 'FallbackIntent',
  'description': 'Default intent when no other intent matches',
  'parentIntentSignature': 'AMAZON.FallbackIntent',
  'lastUpdatedDateTime': datetime.datetime(2021, 9, 4, 17, 38, 49, 849000, tzinfo=tzlocal())},
 {'intentId': 'MXTCT6XS4V',
  'intentName': 'myintentthree',
  'description': 'myintentthree',
  'lastUpdatedDateTime': datetime.datetime(2021, 9, 4, 17, 39, 28, 977000, tzinfo=tzlocal())},
 {'intentId': 'XTHDR1ZCYX',
  'intentName': 'myintentone',
  'description': 'myintentone',
  'lastUpdatedDateTime': datetime.datetime(2021, 9, 4, 17, 39, 23, 908000, tzinfo=tzlocal())},
 {'intentId': 'Y49PYMLVNL',
  'intentName': 'myintenttwo',
  'description': 'myintenttwo',
  'lastUpdatedDateTime': datetime.datetime(2021, 9, 4, 17, 39, 26, 401000, tzinfo=tzlocal())}]

Deleting the Bot

In [55]:
response = client.delete_bot(
    botId='GJNITA86DJ',
    skipResourceInUseCheck=True
)
response

{'ResponseMetadata': {'RequestId': 'f632910a-c217-480a-87d7-e168c9f66d9c',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 04 Sep 2021 11:54:13 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '45',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f632910a-c217-480a-87d7-e168c9f66d9c',
   'x-amz-apigw-id': 'FIuv2EPJIAMF7Uw=',
   'x-amzn-trace-id': 'Root=1-61335e65-0f6659615bfc2f05546f2ef4'},
  'RetryAttempts': 0},
 'botId': 'GJNITA86DJ',
 'botStatus': 'Deleting'}